<a href="https://colab.research.google.com/github/H0wAreU/Statistical-Learning-Labs./blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [236]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [237]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [ ]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [ ]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df["y"].map({'no': 0, 'yes': 1})


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop(['y', 'y_new'],axis=1)

print(y)
x

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y_new, Length: 41188, dtype: int64


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent


In [ ]:
x_train , x_test , y_train , y_test  = train_test_split(x,y,test_size=0.3)

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
31178,48,blue-collar,married,basic.4y,no,no,yes,cellular,may,wed,170,1,999,0,nonexistent
10986,34,blue-collar,married,high.school,no,no,no,telephone,jun,wed,18,6,999,0,nonexistent
7090,37,unemployed,married,university.degree,NaN,no,no,telephone,may,thu,76,3,999,0,nonexistent
36772,55,retired,married,high.school,no,yes,yes,cellular,jun,fri,186,2,999,0,nonexistent
40382,30,admin.,single,university.degree,no,yes,no,cellular,aug,wed,191,1,0,2,success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,38,blue-collar,single,basic.4y,no,NaN,NaN,telephone,may,tue,100,2,999,0,nonexistent
33143,38,services,single,basic.9y,no,no,no,cellular,may,tue,114,1,999,0,nonexistent
20762,31,technician,single,high.school,NaN,no,no,cellular,aug,wed,104,5,999,0,nonexistent
15698,34,admin.,married,high.school,no,yes,yes,cellular,jul,mon,193,1,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [ ]:
num_cols_demo = ['age', 'campaign']
scaler = StandardScaler()

x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo])
x_test[num_cols_demo] = scaler.transform(x_test[num_cols_demo])

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
31178,0.768765,blue-collar,married,basic.4y,no,no,yes,cellular,may,wed,170,-0.565503,999,0,nonexistent
10986,-0.572387,blue-collar,married,high.school,no,no,no,telephone,jun,wed,18,1.248270,999,0,nonexistent
7090,-0.284997,unemployed,married,university.degree,NaN,no,no,telephone,may,thu,76,0.160007,999,0,nonexistent
36772,1.439341,retired,married,high.school,no,yes,yes,cellular,jun,fri,186,-0.202748,999,0,nonexistent
40382,-0.955573,admin.,single,university.degree,no,yes,no,cellular,aug,wed,191,-0.565503,0,2,success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,-0.189201,blue-collar,single,basic.4y,no,NaN,NaN,telephone,may,tue,100,-0.202748,999,0,nonexistent
33143,-0.189201,services,single,basic.9y,no,no,no,cellular,may,tue,114,-0.565503,999,0,nonexistent
20762,-0.859777,technician,single,high.school,NaN,no,no,cellular,aug,wed,104,0.885516,999,0,nonexistent
15698,-0.572387,admin.,married,high.school,no,yes,yes,cellular,jul,mon,193,-0.565503,999,0,nonexistent


Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [ ]:
x_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-2.393039e-16,258.452430,-5.520501e-17,962.700149,0.172245
std,1.000017e+00,261.643723,1.000017e+00,186.353707,0.493194
min,-2.200929e+00,0.000000,-5.655026e-01,0.000000,0.000000
25%,-7.639803e-01,103.000000,-5.655026e-01,999.000000,0.000000
50%,-1.892008e-01,179.000000,-2.027480e-01,999.000000,0.000000
75%,6.729684e-01,319.000000,1.600066e-01,999.000000,0.000000
max,5.558594e+00,4918.000000,1.938600e+01,999.000000,6.000000


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [ ]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')



- **Ordinal Encoding**

In [ ]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [ ]:
ord_col_demo = ['education']

# Impute missing values before encoding
x_train_imputed_ord_demo_df = pd.DataFrame(imputer_ord.fit_transform(x_train[ord_col_demo]), columns=ord_col_demo, index=x_train.index)
x_test_imputed_ord_demo_df = pd.DataFrame(imputer_ord.transform(x_test[ord_col_demo]), columns=ord_col_demo, index=x_test.index)

ordinal_encoder = OrdinalEncoder(categories=[education_categories])
x_train_ord_encoded_demo = ordinal_encoder.fit_transform(x_train_imputed_ord_demo_df)
x_test_ord_encoded_demo = ordinal_encoder.transform(x_test_imputed_ord_demo_df)

# Display x_test (or a processed version of it if needed for context)
x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
31178,0.768765,blue-collar,married,basic.4y,no,no,yes,cellular,may,wed,170,-0.565503,999,0,nonexistent
10986,-0.572387,blue-collar,married,high.school,no,no,no,telephone,jun,wed,18,1.248270,999,0,nonexistent
7090,-0.284997,unemployed,married,university.degree,NaN,no,no,telephone,may,thu,76,0.160007,999,0,nonexistent
36772,1.439341,retired,married,high.school,no,yes,yes,cellular,jun,fri,186,-0.202748,999,0,nonexistent
40382,-0.955573,admin.,single,university.degree,no,yes,no,cellular,aug,wed,191,-0.565503,0,2,success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,-0.189201,blue-collar,single,basic.4y,no,NaN,NaN,telephone,may,tue,100,-0.202748,999,0,nonexistent
33143,-0.189201,services,single,basic.9y,no,no,no,cellular,may,tue,114,-0.565503,999,0,nonexistent
20762,-0.859777,technician,single,high.school,NaN,no,no,cellular,aug,wed,104,0.885516,999,0,nonexistent
15698,-0.572387,admin.,married,high.school,no,yes,yes,cellular,jul,mon,193,-0.565503,999,0,nonexistent


Let's take a look at the imputed and ordinal-encoded `education`.

In [ ]:
print("\nOriginal x_train 'education' head:")
print(x_train[ord_col_demo].iloc[20:25])
print("\nImputed x_train 'education' head (after imputer.transform):")
print(x_train_imputed_ord_demo_df.iloc[20:25])
print("\nOrdinal Encoded x_train 'education' head:")
print(pd.DataFrame(x_train_ord_encoded_demo, columns=ord_col_demo, index=x_train.index).iloc[20:25])


Original x_train 'education' head:
               education
13826        high.school
14846        high.school
2306            basic.6y
12948  university.degree
39860        high.school

Imputed x_train 'education' head (after imputer.transform):
               education
13826        high.school
14846        high.school
2306            basic.6y
12948  university.degree
39860        high.school

Ordinal Encoded x_train 'education' head:
       education
13826        4.0
14846        4.0
2306         2.0
12948        6.0
39860        4.0


**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [190]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train[nom_col_demo])

x_train_imputed_nom_demo = imputer_nom.fit_transform(x_train[nom_col_demo])
x_test_imputed_nom_demo = imputer_nom.fit_transform(x_test[nom_col_demo])

- **Nominal Encoding**

In [ ]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder on the imputed training nominal features (all nominal_features)
x_train_onehot = onehot_encoder.fit_transform(x_train_imputed_nom)

# Transform the imputed test nominal features using the *already fitted* encoder
x_test_onehot = onehot_encoder.transform(x_test_imputed_nom)

x_train_onehot

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 1., 0.]])

In [189]:
# Create a DataFrame from the one-hot encoded training data
onehot_train_df = pd.DataFrame(x_train_onehot,
                               columns=onehot_encoder.get_feature_names_out(nominal_features),
                               index=X_train_copy.index) # Ensure index matches for correct concatenation

# Concatenate X_train_copy with the new one-hot encoded features
X_train_copy = pd.concat([X_train_copy, onehot_train_df], axis=1)

# Drop the original nominal features from X_train_copy
X_train_copy = X_train_copy.drop(columns=nominal_features)

X_train_copy

KeyError: "['marital' 'job' 'default' 'housing' 'loan' 'contact' 'day_of_week'\n 'month' 'poutcome'] not found in axis"

In [ ]:
# Create a DataFrame from the one-hot encoded test data
onehot_test_df = pd.DataFrame(x_test_onehot,
                              columns=onehot_encoder.get_feature_names_out(nominal_features),
                              index=X_test_copy.index) # Ensure index matches for correct concatenation

# Concatenate X_test_copy with the new one-hot encoded features
X_test_copy = pd.concat([X_test_copy, onehot_test_df], axis=1)

# Drop the original nominal features from X_test_copy
X_test_copy = X_test_copy.drop(columns=nominal_features)

X_test_copy

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


1. numerical_features, ordinal_features, and nominal_features

In [238]:
numerical_features = ['age', 'campaign', 'pdays', 'previous']
ordinal_features = ['education']
nominal_features = [ 'job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

In [239]:
X_train_copy = x_train.copy()
X_test_copy = x_test.copy()

In [240]:
import pandas as pd

numerical_features

In [241]:
scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train_copy[numerical_features])
X_test_scaler = scaler.transform(X_test_copy[numerical_features])
#dafaframe
df_train_scaler = pd.DataFrame(X_train_scaler, columns=numerical_features, index=X_train_copy.index)
df_test_scaler = pd.DataFrame(X_test_scaler, columns=numerical_features, index=X_test_copy.index)

ordinal_features

In [242]:
imputer_ord = SimpleImputer(strategy='most_frequent')
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [243]:
imputer_ord = SimpleImputer(strategy='most_frequent')
X_train_imputed_ord = imputer_ord.fit_transform(X_train_copy[ordinal_features])
X_test_imputed_ord = imputer_ord.transform(X_test_copy[ordinal_features])

ordinal_encoder = OrdinalEncoder(categories=[education_categories])
X_train_ord_encoded = ordinal_encoder.fit_transform(X_train_imputed_ord)
X_test_ord_encoded = ordinal_encoder.transform(X_test_imputed_ord)

X_train_ord_encoded_df = pd.DataFrame(X_train_ord_encoded, columns=ordinal_features, index=X_train_copy.index)
X_test_ord_encoded_df = pd.DataFrame(X_test_ord_encoded, columns=ordinal_features, index=X_test_copy.index)

Nominal Encoding

In [244]:

imputer_nom = SimpleImputer(strategy='most_frequent')

# Impute missing values for nominal features in X_train_copy and X_test_copy
X_train_imputed_nom = imputer_nom.fit_transform(X_train_copy[nominal_features])
X_test_imputed_nom = imputer_nom.transform(X_test_copy[nominal_features])

onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform the imputed nominal features for training and test sets
X_train_onehot_encoded = onehot_encoder.fit_transform(X_train_imputed_nom)
X_test_onehot_encoded = onehot_encoder.transform(X_test_imputed_nom)

# Get feature names for the one-hot encoded columns
nominal_feature_names = onehot_encoder.get_feature_names_out(nominal_features)

# Create DataFrames from the one-hot encoded arrays
X_train_onehot_encoded_df = pd.DataFrame(X_train_onehot_encoded, columns=nominal_feature_names, index=X_train_copy.index)
X_test_onehot_encoded_df = pd.DataFrame(X_test_onehot_encoded, columns=nominal_feature_names, index=X_test_copy.index)

# Concatenate new features and drop original nominal features
X_train_copy = pd.concat([X_train_copy, X_train_onehot_encoded_df], axis=1)
X_test_copy = pd.concat([X_test_copy, X_test_onehot_encoded_df], axis=1)

X_train_copy = X_train_copy.drop(columns=nominal_features)
X_test_copy = X_test_copy.drop(columns=nominal_features)


In [245]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

x_train_onehot = onehot_encoder.fit_transform(x_train[nominal_features])
x_test_onehot = onehot_encoder.fit_transform(x_train[nominal_features])
x_train_onehot

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.]])

In [246]:
X_train_processed = pd.concat([df_train_scaler, X_train_ord_encoded_df, X_train_onehot_encoded_df], axis=1)
X_test_processed = pd.concat([df_test_scaler, X_test_ord_encoded_df, X_test_onehot_encoded_df], axis=1)
print("Shape of X_train_processed:", X_train_processed.shape)
print("Shape of X_test_processed:", X_test_processed.shape)

Shape of X_train_processed: (28831, 45)
Shape of X_test_processed: (12357, 45)


In [247]:
# --- Train Logistic Regression ---
model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000) # Increased max_iter for convergence
model.fit(X_train_processed, y_train)

# --- Evaluate the Model ---
y_pred = model.predict(X_test_processed)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Optional: Print accuracy, precision, recall, f1-score separately for clarity
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)

print(f"\nAccuracy: {accuracy:.4f}")
print(f"Average Precision (weighted): {report['weighted avg']['precision']:.4f}")
print(f"Average Recall (weighted): {report['weighted avg']['recall']:.4f}")
print(f"Average F1-score (weighted): {report['weighted avg']['f1-score']:.4f}")



Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     10959
           1       0.69      0.19      0.29      1398

    accuracy                           0.90     12357
   macro avg       0.80      0.59      0.62     12357
weighted avg       0.88      0.90      0.87     12357


Accuracy: 0.8985
Average Precision (weighted): 0.8808
Average Recall (weighted): 0.8985
Average F1-score (weighted): 0.8716
